In [1]:
import os

In [2]:
%pwd

'd:\\NLP\\project\\Text-Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\NLP\\project\\Text-Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.textSummarization.constants import *
from src.textSummarization.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion

In [8]:
import os
import urllib.request as request
import zipfile
from src.textSummarization.logging import logger
from src.textSummarization.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info("f{filename} download! with following info: {headers}")
        else:
            logger.info(f"fFile already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-05 21:38:18,877: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-05 21:38:18,897: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-05 21:38:18,900: INFO: common: creating directory at: artifacts]
[2024-09-05 21:38:18,901: INFO: common: creating directory at: artifacts/data_ingestion]
[2024-09-05 21:38:22,264: INFO: 3658164498: f{filename} download! with following info: {headers}]
